In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
# load dataset containing point geometries of daycare facilities for children
daycares = gpd.read_file('raw/kindertagesstaetten.geojson')

# we want to find the respective plr for each daycare, so we need the plr polygons
plr = gpd.read_file('../plr/plr_only.geojson')

# plr without geometry (for merging later on)
plr_no_geo = plr.drop(columns='geometry').sort_values(by='PLR_ID')

# check for same crs (coordinate reference system)
daycares.crs == plr.crs

True

In [3]:
# Spatial join of daycares and plr
daycares_with_plr = gpd.sjoin(daycares, plr, how='inner', predicate='within')

print("daycares shape:", daycares.shape, "\ndaycares_with_plr shape:", daycares_with_plr.shape)

dacares shape: (2880, 20) 
daycares_with_plr shape: (2880, 23)


In [4]:
daycares_with_plr.head()

,e_nr,e_name,e_bez_nr,e_bez,e_plz,e_ort,e_strasse,e_hnr,e_zusatz,e_tel,...,ang_1,ang_2,t_nr,t_name,t_art,t_web,geometry,index_right,PLR_ID,PLR_NAME
0,01010010,Kita F.A.I.R.play,01,Mitte,10117,Berlin,Albrechtstr.,20,NaN,281 64 73,...,Offene-Arbeit/Situationsansatz,NaN,1224,GFJ - gemeinnützige Gesellschaft für Familien-...,Sonstiger freier Träger,http://www.gfj-berlin.de,POINT (390387.400 5820383.100),448,01100308,Charitéviertel
6,01010130,Kita Pelikan,01,Mitte,10117,Berlin,Friedrichstr.,129,F,2824385,...,Offene-Arbeit/Situationsansatz,NaN,1339,Kinder- und Jugend der Volkssolidarität Berlin...,Sonstiger freier Träger,http://www.volkssolidaritaet-berlin.de,POINT (390390.600 5820745.200),448,01100308,Charitéviertel
8,01010170,Kita Kinderakademie,01,Mitte,10115,Berlin,Habersaathstr.,13,NaN,2824496,...,Offene-Arbeit/Situationsansatz,NaN,1224,GFJ - gemeinnützige Gesellschaft für Familien-...,Sonstiger freier Träger,http://www.gfj-berlin.de,POINT (389784.300 5821358.400),448,01100308,Charitéviertel
27,01010620,Kita Habersaathstraße,01,Mitte,10115,Berlin,Habersaathstr.,5,NaN,282 35 35,...,Situationsansatz,NaN,1312,INA.KINDER.GARTEN gGmbH,Sonstiger freier Träger,http://www.inakindergarten.de,POINT (389720.100 5821392.600),448,01100308,Charitéviertel
42,01011020,Kita CASA AZUL - MAISON BLEUE,01,Mitte,10117,Berlin,Reinhardtstr.,31,NaN,24083164,...,Offene-Arbeit/Situationsansatz/Montessori-Päda...,bilingual deutsch/portugiesisch,7393,CASA AZUL - BLAUES HAUS e.V.,Sonstiger freier Träger,http://www.eukita.de,POINT (390147.400 5820401.400),448,01100308,Charitéviertel


In [5]:
daycares_with_plr.columns

Index(['e_nr', 'e_name', 'e_bez_nr', 'e_bez', 'e_plz', 'e_ort', 'e_strasse',
       'e_hnr', 'e_zusatz', 'e_tel', 'e_web', 'e_platz', 'e_art', 'ang_1',
       'ang_2', 't_nr', 't_name', 't_art', 't_web', 'geometry', 'index_right',
       'PLR_ID', 'PLR_NAME'],
      dtype='object')

In [6]:
# only keep summary relevant features, get rid of geometry -> regular pandas dataframe
# ang_1: pädagogischer Schwerpunkt
# t_art: Trägerart
feature_list = ['PLR_ID','e_name', 'ang_1', 't_art']
daycares_with_plr = daycares_with_plr[feature_list]
daycares_with_plr.head()

,PLR_ID,e_name,ang_1,t_art
0,01100308,Kita F.A.I.R.play,Offene-Arbeit/Situationsansatz,Sonstiger freier Träger
6,01100308,Kita Pelikan,Offene-Arbeit/Situationsansatz,Sonstiger freier Träger
8,01100308,Kita Kinderakademie,Offene-Arbeit/Situationsansatz,Sonstiger freier Träger
27,01100308,Kita Habersaathstraße,Situationsansatz,Sonstiger freier Träger
42,01100308,Kita CASA AZUL - MAISON BLEUE,Offene-Arbeit/Situationsansatz/Montessori-Päda...,Sonstiger freier Träger


In [7]:
# Summarize by groupby
gb = daycares_with_plr.groupby('PLR_ID')

count = gb.size().reset_index(name='kita_count')
names = gb['e_name'].apply(list).reset_index(name='kita_names')
schwerpunkt = gb['ang_1'].value_counts().unstack().reset_index().fillna(0)
traeger = gb['t_art'].value_counts().unstack().reset_index().fillna(0)

In [8]:
# Merge
simple = plr_no_geo \
    .merge(count, on='PLR_ID', how='outer') \
    .merge(names, on='PLR_ID', how='outer')

verbose = plr_no_geo \
    .merge(count, on='PLR_ID', how='outer') \
    .merge(names, on='PLR_ID', how='outer') \
    .merge(schwerpunkt, on='PLR_ID', how='outer') \
    .merge(traeger, on='PLR_ID', how='outer')

In [9]:
simple.columns

Index(['PLR_ID', 'PLR_NAME', 'kita_count', 'kita_names'], dtype='object')

In [10]:
verbose.columns.tolist()

['PLR_ID',
 'PLR_NAME',
 'kita_count',
 'kita_names',
 'Ansatz von Emmi Pikler',
 'Ansatz von Emmi Pikler/Waldorfpädagogik',
 'Freinet-Pädagogik',
 'Freinet-Pädagogik/Ansatz von Emmi Pikler',
 'Freinet-Pädagogik/Offene-Arbeit',
 'Freinet-Pädagogik/Offene-Arbeit/Montessori-Pädagogik/Situationsansatz/Ansatz von Emmi Pikler',
 'Freinet-Pädagogik/Offene-Arbeit/Situationsansatz',
 'Freinet-Pädagogik/Reggio-Pädagogik/Offene-Arbeit/Situationsansatz',
 'Freinet-Pädagogik/Reggio-Pädagogik/Situationsansatz/Ansatz von Emmi Pikler',
 'Freinet-Pädagogik/Situationsansatz/Montessori-Pädagogik',
 'Fröbel-Pädagogik',
 'Fröbel-Pädagogik/Montessori-Pädagogik',
 'Fröbel-Pädagogik/Reggio-Pädagogik/Situationsansatz',
 'Fröbel-Pädagogik/Situationsansatz',
 'Körper und Bewegung/Situationsansatz/Ästhetische Bildung (Musik und Kunst)',
 'Montessori-Pädagogik',
 'Montessori-Pädagogik/Ansatz von Emmi Pikler',
 'Montessori-Pädagogik/Situationsansatz',
 'Montessori-Pädagogik/Situationsansatz/Ansatz von Emmi Pikler'

In [11]:
simple.to_csv('kitas_per_plr_simple.csv', index=False)
verbose.to_csv('kitas_per_plr_verbose.csv', index=False)